# Scraping data
url = https://www.ncwater.org/Water_Supply_Planning/Local_Water_Supply_Plan/report.php?pwsid=03-32-010&year=2016
* Open in Firefox and hit `ctrl-i`
* In the Developer Tools window, activate the `Inspector` tab. 
* Examine the raw HTML code for the portion you want

Tables to scrape
1. System Information
 * Contact info
 * Distribution System
 * Programs
 * Water Conservation
1. Water Use Information
 * Service area
 * Water use by type
 * 


In [40]:
import requests
from bs4 import BeautifulSoup as bs
import io
import pandas as pd
import re

In [2]:
#Create request
baseURL = 'https://www.ncwater.org/Water_Supply_Planning/Local_Water_Supply_Plan/report.php'
params = {'pwsid':'03-320010',
          'year':'2016'
         }

In [4]:
#Retrieve the web page content
html = requests.get(baseURL,params).content

In [5]:
#Convert to a "beautiful soup" object
soup = bs(html, "html5lib")

In [49]:
waterUse = soup.find_all("h2")[1]

In [18]:
#Get the table object containing all the data
divContent = soup.find('div',id="content")
tdMain = divContent.find('td')

In [27]:
#Find the location of the "use-by-type" object
useType = soup.find_all("a",attrs={'name':"use-by-type"})[0]

In [28]:
#Get the 'grand-parent' of the tag and find it's next sibling
dataTable = useType.parent.parent.findNextSibling()

In [21]:
#Show it's contents, to check
dataTable

<table cellspacing="1" class="fancy-table">
	<tbody><tr>
		<th>Type of Use</th>
		<th>Metered<br/>Connections</th>
		<th>Metered<br/>Average Use (MGD)</th>
		<th>Non-Metered<br/>Connections</th>
		<th>Non-Metered<br/>Estimated Use (MGD)</th>
	</tr>
	<tr>
		<td class="left">Residential</td>
		<td>0</td>
		<td>0.0000</td>
		<td>0</td>
		<td>0.0000</td>
	</tr>
	<tr>
		<td class="left">Commercial</td>
		<td>0</td>
		<td>0.0000</td>
		<td>0</td>
		<td>0.0000</td>
	</tr>
	<tr>
		<td class="left">Industrial</td>
		<td>0</td>
		<td>0.0000</td>
		<td>0</td>
		<td>0.0000</td>
	</tr>
	<tr>
		<td class="left">Institutional</td>
		<td>0</td>
		<td>0.0000</td>
		<td>0</td>
		<td>0.0000</td>
	</tr>
	</tbody></table>

In [9]:
#Code to extract data into a dictionary 
rows=dataTable.find_all('tr')

#Create a list of header items 
keyList = []
dataDict = {}

#Convert columns inthe first row to headers
for col in rows[0].find_all('th'):
    colText = col.get_text()
    keyList.append(colText)
    dataDict[colText] = []


#Data rows -> Add to dictionaries
for row in rows[1:]:
    #Get a collection of columns
    cols = row.find_all('td')
    #for i,col in enumerate(cols):
    i=0
    valList = []
    for col in cols:
        #Get the value at the current row/column
        colValue = col.get_text()
        #Get the key from the key list
        key = keyList[i]; i=i+1
        #Get the current values
        valList = dataDict[key]
        #Add the value to the value list
        valList.append(colValue)
        #Update the dictionary's value list
        dataDict[key] = valList

In [10]:
#Convert the dictionary to a data frame
df = pd.DataFrame(dataDict)
df.set_index("Type of Use")

,MeteredAverage Use (MGD),MeteredConnections,Non-MeteredConnections,Non-MeteredEstimated Use (MGD)
Type of Use,,,,
Residential,12.680,"85,294",0,0.000
Commercial,5.860,"5,825",0,0.000
Industrial,1.100,132,0,0.000
Institutional,2.610,"1,355",0,0.000
